In [2]:
!ls /datadisk/tk/encoder_ckpt_20000_OAI.pt

encoder_ckpt_20000_OAI.pt


In [ ]:
eval-acc-llama-oai: ## Eval kb adapter
	python eval.py accuracy \
		--seed 1607 \
		--dataset_dir ${DATASET_DIR} \
		--test_dataset synthetic_data \
		--llm_base_dir '/home/llama_3_8b_ins' \
		--model_dir '/home/kblam_llama_unified/kblam_unified' \
		--encoder_dir '/datadisk/data/gcrbackup/experiments/kblam/exp_v0.1/stage1_lr_0.0001KBTokenLayerFreq3UseExtendedQAMultiEntities2UseOutlier1NoDuplicateKBSizedynamicSepQueryHeadUseDataAugFineTuneQueryKeyFromkey_synthetic_data_OAI_step_20000' \
		--save_dir '/datadisk/data/gcrbackup/experiments/kblam/exp_v0.2_eval' \
		--kb_layer_frequency ${KB_LAYER_FREQ} \
		--ckpt_idx 20000 \
		--lr 0 \
		--kb_size 100 \
		--no-fancy_instruction \
		--encoder_spec oai \
		--llm_type "llama3" \
		--attn_save_dir "/datadisk/kblamatt2" \
		--log_save_dir "/datadisk/kblamatt2/acc_results" 


In [1]:
from experiments.eval import eval_accuracy
from transformers import AutoTokenizer, logging
from kblam.kb_encoder import KBEncoder
from kblam.models.kblam_config import KBLaMConfig
from kblam.models.llama_model import KblamLlamaForCausalLM
from kblam.models.phi3_model import KBLaMPhi3ForCausalLM
from kblam.utils.data_utils import aug_row, generate_multi_entity_qa
from kblam.utils.train_utils import get_kb_embd
import os
import torch
import json
import numpy as np
from datetime import datetime

class ResultsCollector():
    def __init__(
            self,
            dataset_dir,
            encoder_path,
            encoder_spec,
            llm_base_dir,
            llm_type,
            model_path,
            query_head_path,
            test_dataset,
            scale_factor=None,
            kb_layer_frequency=-1):
        self.dataset_dir = dataset_dir
        self.encoder_path = encoder_path
        self.encoder_spec = encoder_spec
        self.llm_base_dir = llm_base_dir
        self.llm_type = llm_type
        self.model_path = model_path
        self.query_head_path = query_head_path
        self.test_dataset = test_dataset
        self.scale_factor = scale_factor

        encoder_model_spec = encoder_spec

        validation_part_start_idx = 120000 if "gpt" in test_dataset else 0
        self.dataset = json.load(open(os.path.join(dataset_dir, test_dataset) + ".json"))[validation_part_start_idx: ]

        self.key_embds = np.load(
            os.path.join(dataset_dir, f"{test_dataset}_{encoder_model_spec}_embd_key.npy")
        ).astype("float32")[validation_part_start_idx: ]
        self.value_embds = np.load(
            os.path.join(dataset_dir, f"{test_dataset}_{encoder_model_spec}_embd_value.npy")
        ).astype("float32")[validation_part_start_idx: ]

        if kb_layer_frequency == -1:
            kb_layer_frequency = 3

        self.tokenizer = AutoTokenizer.from_pretrained(llm_base_dir, trust_remote_code=True, padding_side="left")
        self.tokenizer.pad_token = "^"

        if llm_type == "llama3":
            if self.query_head_path:
                self.model = KblamLlamaForCausalLM.from_pretrained(
                    model_path,
                    device_map="cuda",
                    torch_dtype="auto",
                    trust_remote_code=True,
                )
                self.model.load_query_head(self.query_head_path)
            else:
                self.model = KblamLlamaForCausalLM.from_pretrained(
                    model_path,
                    device_map="cuda",
                    torch_dtype="auto",
                    trust_remote_code=True,
                )
        else:
            self.model = KBLaMPhi3ForCausalLM.from_pretrained(
                model_path,
                device_map="cuda",
                torch_dtype="auto",
                trust_remote_code=True,
            )

        self.model.generation_config.pad_token_id = self.tokenizer.pad_token_id
        self.model.generation_config.eos_token_id = 128009
        self.model.eval()


        kb_config = KBLaMConfig(
            # sep_query_head=True,
            kb_layer_frequency=kb_layer_frequency,
            kb_scale_factor=self.scale_factor,
            **self.model.config.to_dict(),
        )
        self.model.config = kb_config

        self.encoder = KBEncoder(
            encoder_name=encoder_spec.upper(),
            projector_type="linear",
            endpoint_url="",
            out_dim=self.model.config.hidden_size * (self.model.config.num_hidden_layers // kb_layer_frequency + 1),
            frozen_base_model=True,
            projector_kwargs={"mlp_depth": 1, "mlp_hidden_dim": 512},
            device=torch.device("cuda"),
        )

        self.encoder.load_state_dict(torch.load(encoder_path))


    def collect_results(self):
        xs = [50, 100, 200, 400, 800, 1600, 3200, 6400]
        for x in xs:
            for trial in range(5):
                trial_start_time = datetime.now()
                # experiment_name = f"test_synthetic_scale_factor_validation_only_100_{x}_triples_{trial}_trial"
                experiment_name = f"nq_10000_scale_factor_100_{x}_triples_{trial}_trial"
                print(f"starting {experiment_name}")
                eval_accuracy(
                    dataset_dir=self.dataset_dir,
                    test_dataset=self.test_dataset,
                    encoder_spec=self.encoder_spec,
                    kb_scale_factor=self.scale_factor,
                    # encoder_path="/home/t-isazawat/azure-blob/xi-kb-llm/best_ckpt/encoder_ckpt_20000_OAI.pt",
                    encoder_path="/datadisk/tk/encoder_ckpt_20000_OAI.pt",
                    exp_config=experiment_name,
                    fancy_question=False,
                    kb_layer_frequency=3,
                    kb_size=x,
                    # llm_base_dir="/home/t-isazawat/azure-blob/xi-kb-llm/llama3_8b_ins",
                    llm_base_dir=self.llm_base_dir,
                    llm_type="llama3",
                    # model_path="/home/t-isazawat/azure-blob/xi-kb-llm/llama3_8b_ins",
                    model_path=self.model_path,
                    test_batch_size=min(x, 200),
                    use_shift_match=False,
                    # query_head_path="/home/t-isazawat/azure-blob/xi-kb-llm/best_ckpt/learned_query_head_20000_OAI.pth",
                    query_head_path="/datadisk/tk/learned_query_head_20000_OAI.pth",
                    save_dir="/datadisk/kblamatt2",
                    attn_save_dir="/datadisk/kblamatt2",
                    model=self.model,
                    dataset=self.dataset,
                    key_embds=self.key_embds,
                    value_embds=self.value_embds,
                    tokenizer=self.tokenizer,
                    encoder=self.encoder,
                )
                print(f"Took {datetime.now() - trial_start_time}")


/home/msalvaris/miniconda3/envs/kblam/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/msalvaris/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
!pwd

/home/msalvaris/repos/KBLaM


In [6]:
!rm -r /datadisk/kblamatt2/*

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [3]:
!ls /home/llama_3_8b_ins

config.json			  model-00004-of-00004.safetensors
generation_config.json		  model.safetensors.index.json
model-00001-of-00004.safetensors  special_tokens_map.json
model-00002-of-00004.safetensors  tokenizer.json
model-00003-of-00004.safetensors  tokenizer_config.json


In [2]:

# dataset_dir = "/home/t-isazawat/kblam_rebuttal_files/synthetic_perturbed"
# test_dataset = "synthetic_perturbed"
# model_dir = "/home/t-isazawat/azure-blob/xi-kb-llm/llama3_8b_ins"
# base_dir = "/home/t-isazawat/azure-blob/xi-kb-llm/llama3_8b_ins"
# query_head_path = "/home/t-isazawat/azure-blob/xi-kb-llm/best_ckpt/learned_query_head_20000_OAI.pth"
# encoder_path = "/home/t-isazawat/azure-blob/xi-kb-llm/best_ckpt/encoder_ckpt_20000_OAI.pt"

# dataset_dir = "/datadisk/tk/dataset"
# test_dataset = "gpt_data"

dataset_dir = "/datadisk/data/gcrbackup/oai_embd"
test_dataset = "synthetic_data"

model_dir = "/kblam_llama_unified/kblam_unified"
base_dir = "/datadisk/tk/llama3_8b_ins"
query_head_path = "/datadisk/tk/learned_query_head_20000_OAI.pth"
# query_head_path = None
encoder_path = "/datadisk/tk/encoder_ckpt_20000_OAI.pt"

collector = ResultsCollector(
    dataset_dir,
    encoder_path,
    "oai",
    base_dir,
    "llama3",
    base_dir,
    query_head_path,
    test_dataset,
    scale_factor=100
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


BASE MODEL: /datadisk/tk/llama3_8b_ins


Loading checkpoint shards:   0%|                                                                                                                        | 0/4 [00:00<?, ?it/s]/home/msalvaris/miniconda3/envs/kblam/lib/python3.10/site-packages/torch/nn/modules/module.py:2400: UserWarning: for model.embed_tokens.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/home/msalvaris/miniconda3/envs/kblam/lib/python3.10/site-packages/torch/nn/modules/module.py:2400: UserWarning: for model.layers.0.self_attn.q_proj.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copy

Learned query heads loaded.


/tmp/ipykernel_2651083/3584050049.py:103: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.encoder.load_state_dict(torch.load(encoder_path))


In [3]:
collector.collect_results()

starting nq_10000_scale_factor_100_50_triples_0_trial
SAVING /datadisk/kblamatt2 /datadisk/kblamatt2/nq_10000_scale_factor_100_50_triples_0_trial_0.npy
SAVING /datadisk/kblamatt2 /datadisk/kblamatt2/nq_10000_scale_factor_100_50_triples_0_trial_1.npy
SAVING /datadisk/kblamatt2 /datadisk/kblamatt2/nq_10000_scale_factor_100_50_triples_0_trial_2.npy
SAVING /datadisk/kblamatt2 /datadisk/kblamatt2/nq_10000_scale_factor_100_50_triples_0_trial_3.npy
SAVING /datadisk/kblamatt2 /datadisk/kblamatt2/nq_10000_scale_factor_100_50_triples_0_trial_4.npy
SAVING /datadisk/kblamatt2 /datadisk/kblamatt2/nq_10000_scale_factor_100_50_triples_0_trial_5.npy
SAVING /datadisk/kblamatt2 /datadisk/kblamatt2/nq_10000_scale_factor_100_50_triples_0_trial_6.npy
SAVING /datadisk/kblamatt2 /datadisk/kblamatt2/nq_10000_scale_factor_100_50_triples_0_trial_7.npy
SAVING /datadisk/kblamatt2 /datadisk/kblamatt2/nq_10000_scale_factor_100_50_triples_0_trial_8.npy
SAVING /datadisk/kblamatt2 /datadisk/kblamatt2/nq_10000_scale_fa

KeyboardInterrupt: 

In [ ]:
len(collector.dataset)

29096

In [ ]:
os.environ['SEP_QUERY_HEAD'] = 'TRUE'
os.environ['LENGTH_INVARIANCE'] = ''
# os.environ['SEP_QUERY_HEAD'] = ''
encoder_model_spec = 'OAI'
# train_dataset_name = 'avocado_new'
train_dataset_name = 'gpt_data'
epoch = 10000
# lr = 0.0005
lr = 0.0001
extended_qa_spec = 'UseExtendedQA'
outlier_spec = "UseOutlier1"
# outlier_spec = ""
multi_entity_string = "MultiEntities2"
# multi_entity_string = ""
# outlier_spec = ""
kb_size_spec = 'KBSizedynamic'
# kb_size_spec = 'KBSize50'
# os.environ['SCALE_FACTOR'] = '40'
os.environ['SCALE_FACTOR'] = ''
duplicate_spec = "NoDuplicate"
kb_layer_frequency = 1
kb_layer_frequency_str = f'KBTokenLayerFreq{kb_layer_frequency}'
# kb_layer_frequency_str=''
# outlier_spec = ''
key_src = 'key'

llm_model_spec = '/home/t-wangx/llama_weights/llama3_8b_ins'
print(f'/home/t-wangx/azure_blob/xi-kb-llm/outputs/ckpts/stage1_0__lr_{lr}{kb_layer_frequency_str}{extended_qa_spec}{multi_entity_string}{outlier_spec}{duplicate_spec}{kb_size_spec}SepQueryHeadUseDataAugKeyFrom{key_src}_{encoder_model_spec}_{train_dataset_name}_llama3_epoch_{epoch}')
print( f'/home/t-wangx/azure_blob/xi-kb-llm/outputs/ckpts/stage1_0__lr_{lr}{kb_layer_frequency_str}{extended_qa_spec}{multi_entity_string}{outlier_spec}{duplicate_spec}{kb_size_spec}SepQueryHeadUseDataAugFineTuneQueryKeyFrom{key_src}_{train_dataset_name}_{encoder_model_spec}_epoch_{epoch}')


/home/t-wangx/azure_blob/xi-kb-llm/outputs/ckpts/stage1_0__lr_0.0001KBTokenLayerFreq1UseExtendedQAMultiEntities2UseOutlier1NoDuplicateKBSizedynamicSepQueryHeadUseDataAugKeyFromkey_OAI_gpt_data_llama3_epoch_10000
/home/t-wangx/azure_blob/xi-kb-llm/outputs/ckpts/stage1_0__lr_0.0001KBTokenLayerFreq1UseExtendedQAMultiEntities2UseOutlier1NoDuplicateKBSizedynamicSepQueryHeadUseDataAugFineTuneQueryKeyFromkey_gpt_data_OAI_epoch_10000
